In [1]:
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.feature_selection import mutual_info_classif, SelectKBest, SelectFromModel
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, \
    r2_score, mean_absolute_error, mean_squared_error, median_absolute_error
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, \
    GradientBoostingClassifier, AdaBoostClassifier
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression, LassoCV
from sklearn.svm import SVC
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats as stats
from sklearn.feature_selection import RFE, RFECV
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import RandomizedSearchCV


In [3]:
path = "C:\\Users\\ritaf\\Documents\\ESTUDOS\\NOVA IMS\\3º Ano\\Capstone Project\\Projeto\\"
train_orig = pd.read_csv(path + 'dados_ml.csv')
customer = pd.read_csv(path + 'customer_data.csv')

In [12]:
features = ['AppointmentWeekNumber', 'AppointmentDayOfMonth', 'AppointmentHour', 'WeekendConsults', 'WeekdayConsults', 
            'Adults', 'Children', 'Babies', 'AffiliatedPatient', 'PreviousAppointments', 'PreviousNoShows', 
            'DaysUntilConfirmation', 'OnlineBooking', 'AppointmentChanges', 'BookingToConsultDays', 'ParkingSpaceBooked', 
            'SpecialRequests', 'NoInsurance', 'ExtraExamsPerConsult', 'DoctorAssigned', 'ConsultPriceEuros', 
            '%PaidinAdvance', 'CountryofOriginHDI (Year-1)']

train = train_orig.loc[:, features]

days until confirmation - last time minutes late

In [18]:
train.describe().T

,count,mean,std,min,25%,50%,75%,max
AppointmentWeekNumber,13938.0,28.851413,13.901656,1.000,17.0000,29.000,41.000,53.000
AppointmentDayOfMonth,13938.0,15.888148,8.850454,1.000,8.0000,16.000,24.000,31.000
AppointmentHour,13938.0,18.874121,2.884563,14.000,16.2500,19.000,21.250,23.750
WeekendConsults,13938.0,0.882695,0.945422,0.000,0.0000,1.000,2.000,14.000
WeekdayConsults,13938.0,2.292510,1.735639,0.000,1.0000,2.000,3.000,35.000
Adults,13938.0,1.885852,0.547217,0.000,2.0000,2.000,2.000,4.000
Children,13938.0,0.141627,0.452717,0.000,0.0000,0.000,0.000,3.000
Babies,13938.0,0.006959,0.116244,0.000,0.0000,0.000,0.000,10.000
AffiliatedPatient,13938.0,0.005022,0.070692,0.000,0.0000,0.000,0.000,1.000
PreviousAppointments,13938.0,0.208495,2.079191,0.000,0.0000,0.000,0.000,53.000


In [19]:
train['WeekendConsults'].value_counts()

WeekendConsults
0     6058
1     3946
2     3715
3       84
4       78
5       21
6       20
8        8
7        4
9        2
14       2
Name: count, dtype: int64

In [20]:
train['WeekdayConsults'].value_counts()

WeekdayConsults
1     3858
2     3801
3     2922
4     1268
0     1010
5      765
6       87
7       55
10      51
8       40
9       28
11      12
14       7
15       6
21       4
18       4
20       4
13       4
12       3
16       3
22       2
17       1
19       1
34       1
35       1
Name: count, dtype: int64

In [17]:
train['DoctorAssigned'].value_counts()

DoctorAssigned
 6    8609
 3    3544
 5     544
 1     531
 2     474
 0     144
-1      62
 4      30
Name: count, dtype: int64

In [15]:
customer.set_index(['PatientID'], inplace = True)
customer

,FirstName,LastName,Email,Age,Username
PatientID,,,,,
1,John,Doe,johndoe@email.com,35,johndoe35
2,Jane,Smith,janesmith@email.com,28,janesmith28
3,Robert,Johnson,robertjohnson@email.com,42,robjohn42
4,Alice,Williams,alicewilliams@email.com,31,alicewilliams31
5,Michael,Brown,michaelbrown@email.com,45,mikebrown45
...,...,...,...,...,...
146,Karen,Martin,karenmartin@email.com,45,karenm45
147,Joseph,Anderson,josephanderson@email.com,36,josepha36
148,Laura,Perez,lauraperez@email.com,31,laurap31


In [16]:
customer.duplicated().sum()

0

In [18]:
print(customer['Username'].duplicated().sum())
print(customer['Email'].duplicated().sum())

0
0
